In [ ]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
temp = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Dataset/merged_training.pkl')

In [ ]:
temp

,text,emotions
27383,i feel awful about it too because it s my job ...,sadness
110083,im alone i feel awful,sadness
140764,ive probably mentioned this before but i reall...,joy
100071,i was feeling a little low few days back,sadness
2837,i beleive that i am much more sensitive to oth...,love
...,...,...
566,that was what i felt when i was finally accept...,joy
36236,i take every day as it comes i m just focussin...,fear
76229,i just suddenly feel that everything was fake,sadness
131640,im feeling more eager than ever to claw back w...,joy


In [ ]:
data_90, data_001= train_test_split(temp, test_size=0.01, stratify = temp['emotions'], random_state = 42)

In [ ]:
data_001

,text,emotions
15557,i feel like cinderalla kena tortured by my ste...,fear
55740,im so damn scared of hurting peoples feelings ...,anger
16639,i decided to do a few warm up runs up and down...,sadness
135510,im no expert on cars but id love it if someone...,joy
63516,i forget this little address of mine and then ...,sadness
...,...,...
6682,i dont know why i am so stuck on that feeling ...,love
89552,i invest most of my feelings in bands that i l...,joy
31911,i feel better again,joy
84162,i often feel like im doomed to be single forev...,sadness


In [ ]:
data_001.emotions.value_counts()

joy         1411
sadness     1212
anger        573
fear         477
love         346
surprise     150
Name: emotions, dtype: int64

In [ ]:
data_001.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/carer_0.01_percent.csv', index=True,header=True)

In [ ]:
temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/carer_0.01_percent.csv')

del temp['Unnamed: 0']
temp


,text,emotions
0,i feel like cinderalla kena tortured by my ste...,fear
1,im so damn scared of hurting peoples feelings ...,anger
2,i decided to do a few warm up runs up and down...,sadness
3,im no expert on cars but id love it if someone...,joy
4,i forget this little address of mine and then ...,sadness
...,...,...
4164,i dont know why i am so stuck on that feeling ...,love
4165,i invest most of my feelings in bands that i l...,joy
4166,i feel better again,joy
4167,i often feel like im doomed to be single forev...,sadness


In [ ]:
temp.emotions.value_counts()

joy         1411
sadness     1212
anger        573
fear         477
love         346
surprise     150
Name: emotions, dtype: int64

In [ ]:
max = 0
sum = 0
for sentence in temp['text']:
  val = len(list(sentence.split(" ")))
  sum +=val
  if max < val:
    max = val

max

78

In [ ]:
sum/4169

19.18541616694651

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(temp['text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

list_tokenized_train = tokenizer.texts_to_sequences(temp['text'])
  
  
MAX_LEN = 30  
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train = temp['emotions']


X_train, X_test = train_test_split(X_train, test_size=0.2,  random_state = 42)
y_train, y_test = train_test_split(y_train, test_size=0.2,  random_state = 42)

encoding = {
    'joy': 0,
    'sadness': 1,
    'anger': 2,
    'fear': 3,
    'love': 4,
    'surprise': 5
}

y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(3335, 30)
(834, 30)
(3335, 6)
(834, 6)


In [ ]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/Colab Notebooks/Dataset/cc.en.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,trainable=True,
                          weights = [embedd_matrix])(seq_input)


  cnn = keras.layers.Conv1D(cnn_1_unit,3)(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((28,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.GRU(lstm_unit,return_sequences=True,input_shape =(28,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


clr_step_size = int((len(X_train)/16))
base_lr = 2e-5
max_lr = 1e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr , step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 30,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train,y = y_train,epochs = 20, batch_size = 32,callbacks = [stop], 
             validation_data = (X_test,y_test))


tuner.results_summary()

Trial 30 Complete [00h 02m 14s]
val_accuracy: 0.371702641248703

Best val_accuracy So Far: 0.8465228080749512
Total elapsed time: 02h 18m 39s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1632293025/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 48
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 224
lstm_dropout: 0.2
cnn_2_unit: 96
cnn_2_dropout: 0.30000000000000004
Score: 0.8465228080749512
Trial summary
Hyperparameters:
attention_unit: 96
cnn_1_unit: 16
cnn_1_dropout: 0.1
lstm_unit: 160
lstm_dropout: 0.5
cnn_2_unit: 160
cnn_2_dropout: 0.4
Score: 0.8225419521331787
Trial summary
Hyperparameters:
attention_unit: 112
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 192
lstm_dropout: 0.2
cnn_2_unit: 160
cnn_2_dropout: 0.5
Score: 0.8153477311134338
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 80
cnn_1_dropout: 0.2
lstm_unit: 256
lstm_dropout: 0.2
cnn_2_unit: 160
cnn_2_dropo